In [1]:
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from tqdm import tqdm
import pandas as pd
import numpy as np

# Basic imports
import pandas as pd
import numpy as np
import logging
import warnings
import os
from collections import Counter
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
# PyTorch imports
import torch

# PyTorch Tabular and Model imports (replace with correct imports based on your package structure)
from pytorch_tabular import TabularModel
from pytorch_tabular.models.category_embedding import CategoryEmbeddingModel

from pytorch_tabular.categorical_encoders import CategoricalEmbeddingTransformer
import sys
sys.path.append('../')
from src.data.process_data import load_dataset, split_dataset

# Suppress common warnings
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)


In [4]:
data_path = '../data/'
drug_syn_path = os.path.join(data_path, 'drug_synergy.csv')
cell_lines_path = os.path.join(data_path, 'cell_lines.csv')
drug_portfolio_path = os.path.join(data_path, 'drug_portfolio.csv')
smiles_path = os.path.join(data_path, 'drug_portfolio_smiles.csv')

full_dataset_df, column_type_dict = load_dataset(drug_syn_path, cell_lines_path, drug_portfolio_path, 
                                                 smiles_path=smiles_path
                                                 )

  0%|          | 0/118 [00:00<?, ?it/s][17:41:40] SMILES Parse Error: syntax error while parsing: Antibody
[17:41:40] SMILES Parse Error: check for mistakes around position 1:
[17:41:40] Antibody
[17:41:40] ^
[17:41:40] SMILES Parse Error: Failed parsing SMILES 'Antibody' for input: 'Antibody'
[17:41:40] SMILES Parse Error: syntax error while parsing: CC(=O)N1CCN(CCOc2ccc(cc2)C3CCN(CC3)C4=Nn5c(CC4)nnc5C(F)(F)F)CC1;CC(=O)N1CCN(CC1)CCOc2ccc(cc2)C3CCN(CC3)C4=Nn5c(nnc5C(F)(F)F)CC4
[17:41:40] SMILES Parse Error: check for mistakes around position 64:
[17:41:40] CC4)nnc5C(F)(F)F)CC1;CC(=O)N1CCN(CC1)CCOc
[17:41:40] ~~~~~~~~~~~~~~~~~~~~^
[17:41:40] SMILES Parse Error: Failed parsing SMILES 'CC(=O)N1CCN(CCOc2ccc(cc2)C3CCN(CC3)C4=Nn5c(CC4)nnc5C(F)(F)F)CC1;CC(=O)N1CCN(CC1)CCOc2ccc(cc2)C3CCN(CC3)C4=Nn5c(nnc5C(F)(F)F)CC4' for input: 'CC(=O)N1CCN(CCOc2ccc(cc2)C3CCN(CC3)C4=Nn5c(CC4)nnc5C(F)(F)F)CC1;CC(=O)N1CCN(CC1)CCOc2ccc(cc2)C3CCN(CC3)C4=Nn5c(nnc5C(F)(F)F)CC4'
[17:41:40] SMILES Parse Error: syntax 

Failed to convert 23/118 SMILES strings to Morgan fingerprints
Setting them to all-zero arrays


In [12]:
print(full_dataset_df.columns[column_type_dict['numerical']['col_idx'][-1]+1])


Synergy score


In [9]:
column_type_dict['numerical']['col_idx'][-1]

4115

In [66]:
full_dataset_df_smiles = full_dataset_df.copy()
fpSize=2048
fp_columns_a = [f'MorganFP_A_{i}' for i in range(fpSize)]
fp_columns_b = [f'MorganFP_B_{i}' for i in range(fpSize)]

full_dataset_df_smiles[fp_columns_a] = full_dataset_df_smiles['Compound A'].map(morgan_fp_dict).apply(pd.Series)
full_dataset_df_smiles[fp_columns_b] = full_dataset_df_smiles['Compound B'].map(morgan_fp_dict).apply(pd.Series)


In [ ]:
column_type_dict['numerical']['col_names'] += fp_columns_a + fp_columns_b

In [96]:
column_type_dict['numerical']['col_idx']

[12, 13, 14, 15, 16, 17, 18, 19]

In [94]:
column_type_dict['numerical']['col_idx'] += list(range(column_type_dict['numerical']['col_idx'][-1]+1, column_type_dict['numerical']['col_idx'][-1]+1+2*fpSize))
column_type_dict['numerical']['col_idx'] 

[12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 1